In [1]:
from __future__ import annotations

from torch.utils.data import random_split

from mmpfn.datasets.petfinder_pawpularity import PawpularityDataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import r2_score, root_mean_squared_error
from mmpfn.models.mmpfn import MMPFNRegressor
from mmpfn.models.mmpfn.constants import ModelInterfaceConfig
from mmpfn.models.mmpfn.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune_mm.finetune_tabpfn_main import fine_tune_mmpfn

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # for debugging

In [3]:
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/pawpularity")
dataset = PawpularityDataset(data_path)
# _ = dataset.get_images()
_ = dataset.get_embeddings()

Load embeddings from embeddings/pawpularity/pawpularity.pt


In [4]:
r2_scores = []
for seed in range(5):
    torch.manual_seed(seed)
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
    image_train = train_dataset.dataset.embeddings[train_dataset.indices]#.unsqueeze(1)
    image_test = test_dataset.dataset.embeddings[test_dataset.indices]#.unsqueeze(1)
        
    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_mmpfn_airbnb.ckpt"

    fine_tune_mmpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="rmse",
        # Input Data
        X_train=pd.DataFrame(X_train),
        image_train=image_train,
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="regression",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
        freeze_input=True,  # Freeze the input layers (encoder and y_encoder) during finetuning
        mixer_type='MGM', # MGM MGM+CQAM
        mgm_heads=8,
        cqam_heads=4,
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = MMPFNRegressor(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
        mixer_type='MGM', # MGM MGM+CQAM
        mgm_heads=8,
        cqam_heads=4,
    )

    clf_finetuned = model_finetuned.fit(X_train, image_train, y_train)
    r2 = root_mean_squared_error(y_test, clf_finetuned.predict(X_test, image_test))
    print("r2 (Finetuned):", r2)
    r2_scores.append(r2)

Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-12 00:47:08,209] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   2%|▏         | 2/100 [00:02<04:06,  2.52s/it, Best Val. Loss=0.202, Best Val. Score=-0.202, Training Loss=1.32, Val. Loss=0.202, Patience=50, Utilization=0, Grad Norm=inf][2025-09-12 00:47:10,584] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  17%|█▋        | 17/100 [00:48<04:14,  3.07s/it, Best Val. Loss=0.2, Best Val. Score=-0.2, Training Loss=0.078, Val. Loss=0.224, Patience=36, Utilization=0, Grad Norm=21.1]  [2025-09-12 00:47:56,509] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [05:09,  3.09s/it, Best Val. Loss=0.2, Best Val. Score=-0.2, Training Loss=-0.65, Val. Loss=0.211, Patience=-47, Utilization=0, Grad Norm=10.7]                          
[2025-09-12 00:52:14,899] INFO - Initial Validation Loss: 0.20205622911453247 Best Valida

r2 (Finetuned): 0.2095202024956909


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-12 00:52:25,815] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   2%|▏         | 2/100 [00:02<04:52,  2.98s/it, Best Val. Loss=0.205, Best Val. Score=-0.205, Training Loss=1.35, Val. Loss=0.205, Patience=50, Utilization=0, Grad Norm=inf]


KeyboardInterrupt: 

In [ ]:
# get mean and std of accuracy scores
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)
print("Mean R2:", mean_r2)
print("Std R2:", std_r2)